In [ ]:
%%capture
%pip install -U 'rockfish[labs]' -f 'https://docs142.rockfish.ai/packages/index.html'

In [ ]:
import rockfish as rf
import rockfish.actions as ra

In [ ]:
# download our example of timeseries data: finance.csv
!wget --no-clobber https://docs142.rockfish.ai/tutorials/finance.csv

--2024-06-07 14:58:01--  https://docs142.rockfish.ai/tutorials/finance.csv
Resolving docs142.rockfish.ai (docs142.rockfish.ai)... 18.154.206.84, 18.154.206.67, 18.154.206.70, ...
Connecting to docs142.rockfish.ai (docs142.rockfish.ai)|18.154.206.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3444556 (3.3M) [text/csv]
Saving to: ‘finance.csv’

finance.csv         100%[===================>]   3.28M  --.-KB/s    in 0.1s    

2024-06-07 14:58:03 (23.1 MB/s) - ‘finance.csv’ saved [3444556/3444556]



In [ ]:
# Let's have 100 rows of finance data as an example
dataset = rf.Dataset.from_csv("finance", "finance.csv")
dataset.table = dataset.table.slice(0,100)
dataset.to_pandas()

,customer,age,gender,merchant,category,amount,fraud,timestamp
0,C1093826151,4,M,M348934600,transportation,4.55,0,2023-01-01
1,C575345520,2,F,M348934600,transportation,76.67,0,2023-01-01
2,C1787537369,2,M,M1823072687,transportation,48.02,0,2023-01-01
3,C1732307957,5,F,M348934600,transportation,55.06,0,2023-01-01
4,C842799656,1,F,M348934600,transportation,25.62,0,2023-01-01
...,...,...,...,...,...,...,...,...
95,C949319027,2,M,M855959430,hyper,44.72,0,2023-01-01
96,C859628493,3,F,M348934600,transportation,17.17,0,2023-01-01
97,C1641076533,2,F,M1823072687,transportation,9.69,0,2023-01-01
98,C454239699,3,F,M348934600,transportation,7.86,0,2023-01-01


In [ ]:
conn = rf.Connection.local()

### Replace the low ranking categories with a new value

For example, in the Column "category", it has 8 categeries. We can keep top 3 frequent categories and replace other categories by "others".

In [ ]:
# take a look the frequences for 8 categories
dataset.to_pandas()["category"].value_counts()

category
transportation        80
wellnessandbeauty      4
health                 4
barsandrestaurants     3
food                   3
hyper                  3
fashion                2
hotelservices          1
Name: count, dtype: int64

In [ ]:
# define the method, selected column, options for the action
method = "replace"
select_col = "category"
options = {"top_k": 3, "replaced_value": "others"}
# create an action
replace_transform = ra.Transform(
    {"function": {"handle_high_cardinality": [method, select_col, options]}}
)

In [ ]:
save = rf.actions.DatasetSave({"name": "replaced_dataset"})
preprocess_builder = rf.WorkflowBuilder()
preprocess_builder.add_path(dataset, replace_transform, save)
preprocess_workflow = await preprocess_builder.start(conn)

replaced_dataset = None
async for sds in preprocess_workflow.datasets():
    replaced_dataset = await sds.to_local(conn)
# check the result
replaced_dataset.to_pandas()["category"].value_counts()

category
transportation       80
others               12
wellnessandbeauty     4
health                4
Name: count, dtype: int64

### Truncate the low frequent categories
After replacing the low frequent categories by a defined value, we can further use the `amplify` action to drop the records matching this replaced value.

In [ ]:
# use the previous replace action
method = "replace"
select_col = "category"
options = {"top_k": 3, "replaced_value": "others"}
replace_transform = ra.Transform(
    {"function": {"handle_high_cardinality": [method, select_col, options]}}
)
# amplify action
amplify = ra.PostAmplify(
    {
        "query_ast": {
            "eq": [select_col, "others"],
        },
        "drop_match_percentage": 1.0,
        "drop_other_percentage": 0.0,
    }
)

In [ ]:
save = rf.actions.DatasetSave({"name": "truncated_dataset"})
preprocess_builder = rf.WorkflowBuilder()
preprocess_builder.add_path(dataset, replace_transform, amplify, save)
preprocess_workflow = await preprocess_builder.start(conn)

truncated_dataset = None
async for sds in preprocess_workflow.datasets():
    truncated_dataset = await sds.to_local(conn)
# check the result
truncated_dataset.to_pandas()["category"].value_counts()

category
transportation       80
wellnessandbeauty     4
health                4
Name: count, dtype: int64